In [1]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass()

from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

 ········


In [2]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

In [3]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [4]:
config = {"configurable": {"session_id": "abc2"}}

In [18]:
BASE_PRICE = 100
MINIMUM_PRICE = 70

In [6]:
def query_bot(prompt: str) -> str:
    response = model([HumanMessage(content=prompt)])
    return response[0].content

In [7]:
def handle_user_response(user_input, bot_offer):
    if user_input.lower() == "accept":
        print(f"Bot: Great! You've accepted the offer at ${bot_offer}.")
        return True
    
    elif user_input.lower() == "reject":
        print("Bot: Sorry to hear that. The negotiation has ended.")
        return True
    
    elif user_input.lower() == "counteroffer":
        return False
    
    else:
        print("Invalid option. Please type 'accept', 'reject', or 'counteroffer'.")
        return False

In [25]:
def handle_negotiation():
    bot_offer = BASE_PRICE
    print(f"Bot: I’m offering the product for ${BASE_PRICE}. What would you like to do? (Type 'accept', 'reject', or 'counteroffer')")

    while True:
        user_action = input("You (Customer): ")

        negotiation_over = handle_user_response(user_action, bot_offer)
        if negotiation_over:
            break
    
        while not negotiation_over:
            user_input = input("You (Customer): Please enter your counteroffer: ")

            if user_input.lower() == "exit":
                print("You have exited the negotiation.")
                return

            try:
                user_offer = float(user_input)
            except ValueError:
                print("Invalid offer. Please provide a valid number.")
                continue

            if user_offer >= bot_offer or user_offer >= MINIMUM_PRICE:
                print(f"Bot: Offer accepted at ${user_offer}!")
                return
            elif bot_offer == MINIMUM_PRICE:
                print(f"Bot: ${MINIMUM_PRICE} is my minimimum offer, you can accept or reject it.")
                break
            elif (bot_offer - 10.0) == MINIMUM_PRICE:
                print(f"Bot: ${MINIMUM_PRICE} is my minimimum offer, you can accept or reject it.")
                bot_offer = bot_offer - 10
                break
            else:
                print(f"Bot: Unfortunately, we cannot accept your offer of ${user_offer}.")
                bot_offer = max(bot_offer - 10.0, MINIMUM_PRICE)
                print(f"Bot: I can offer you ${bot_offer}. Would you like to accept, reject, or counteroffer?")
                break

handle_negotiation()

Bot: I’m offering the product for $100. What would you like to do? (Type 'accept', 'reject', or 'counteroffer')


You (Customer):  counteroffer
You (Customer): Please enter your counteroffer:  80


Bot: Offer accepted at $80.0!
